In [1]:
# If running on Google Colab, only cleverhans needs installation. This can be done via:
# !pip install cleverhans

# If running locally, we've listed (TODO) our dependencies in requirements.txt, so the following
# should get everything up and running:
# !pip install -r requirements.txt

import numpy
import keras
import pandas
import requests
import io
import zipfile
import os
import re
import cleverhans
import tensorflow

from cleverhans.attacks import FastGradientMethod
from cleverhans.attacks import CarliniWagnerL2
from cleverhans.attacks import SaliencyMapMethod
from cleverhans.attacks_tf import jacobian_augmentation
from cleverhans.attacks_tf import jacobian_graph
from cleverhans.loss import CrossEntropy
from cleverhans.train import train
from cleverhans.utils_keras import KerasModelWrapper
from cleverhans.utils_tf import model_eval

from keras.models import Sequential
from keras.layers import Dense

numpy.random.seed(0xC0FFEE)
tensorflow.set_random_seed(0xC0FFEE)
rng = numpy.random.RandomState(0xC0FFEE)

Using TensorFlow backend.



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



# Dataset

## Loading data

Run the below code to download a copy of the dataset (if you don't already have it):

In [0]:
response = requests.get("http://www.schonlau.net/masquerade/masquerade-data.zip")

dataset_file = io.BytesIO(response.content)

zipped_dataset = zipfile.ZipFile(dataset_file)
zipped_dataset.extractall('data/masquerade-data')

In [0]:
# http://www.schonlau.net/intrusion.html
# download Masquerade Data (zip File)

import pandas as pd
directory = './data/masquerade-data'

In [0]:
def sorted_nicely( l ):
    """ Sorts the given iterable in the way that is expected.
 
    Required arguments:
    l -- The iterable to be sorted.
 
    """
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key = alphanum_key)

In [0]:
users = range(1,51)
df = pd.DataFrame()

for filename in sorted_nicely(os.listdir(directory)):
    user = pd.read_csv(os.path.join(directory, filename), header=None)
    df = pd.concat([df, user], axis = 1)
    
df.columns = sorted_nicely(os.listdir(directory))

We've loaded in the dataset, but need to do a little co-ercion to get it how we need. Firstly, make sure that all the values in this dataframe are categorical variables which share the same data type:

In [0]:
commands = numpy.unique(df)
command_dtype = pandas.api.types.CategoricalDtype(commands)

for column in df:
    df[column] = df[column].astype(command_dtype)

In [0]:
labelled, unlabelled = df.head(5000), df.tail(len(df) - 5000)  # ignore unlabeled

Plan is convert to the following format:

  user, command1?, command2?, ..., 
  
 so the first column is a label, and the second a one-hot encoding of the command.
 
 When we do the rolling window aggregation, we just sum the columns (per-user).
 
 Use [rolling window sampling](https://pcp.io/books/PCP_PG/html/LE42586-PARENT.html).

In [8]:
def rolling_window_command_counts(commands, window_size):
    
    # Save a copy the name of the series to add again to our output. This will preserve the mapping of
    # user identifier to (it's column header in the dataframe it came from), which in
    # this case is the user identifier. 
    user = commands.name

    # Convert the single column "which command was run?" to a column for each
    # command, which says "was command <x> run?"
    commands = pandas.get_dummies(commands)

    # Take a rolling sample of the last 100 commands, then sum each "was command <x> run?"
    # columns to give a bunch "command <x> was run <y> times in this window".
    command_counts = commands.rolling(window=window_size).aggregate(numpy.sum)

    # Remove the first 100 rows because they contain data from blocks of size < 100.
    command_counts = command_counts[window_size-1:]
    
    # Preserve the user identifier (see top of function) as a new column:
    
    # First, a nasty hack: https://github.com/pandas-dev/pandas/issues/19136
    command_counts = command_counts.rename(columns=str)  
    
    # Then, add in the user (with an adhoc parser to turn the label into a number)
    command_counts['user'] = int(user.replace('User', ''))

    return command_counts

# Example
rolling_window_command_counts(labelled['User1'], 100)

,%backup%,.java_wr,.maker_w,.wrapper,.xinitrc,.xsessio,1.1,1.2,1.3,4Dwm,...,xxx,yacc,ypcat,yppasswd,z,zip,zsh,zubs,zz2,user
99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
105,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
106,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
108,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [9]:
labelled_dataset = pandas.concat([
        rolling_window_command_counts(commands, 100)
        for user, commands in labelled.iteritems()
    ],
    ignore_index=True,  # reset index to go from 0 to 4900
)

labelled_dataset

,%backup%,.java_wr,.maker_w,.wrapper,.xinitrc,.xsessio,1.1,1.2,1.3,4Dwm,...,xxx,yacc,ypcat,yppasswd,z,zip,zsh,zubs,zz2,user
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [0]:
labels = labelled_dataset['user'] - 1
dataset = labelled_dataset.drop(columns=['user'])

In [0]:
labels =  keras.utils.to_categorical(labels, num_classes=50)

In [0]:
from sklearn.model_selection import train_test_split

training_data, testing_data, training_labels, testing_labels = train_test_split(
    dataset,
    labels, 
    test_size=0.10,
#     stratify=list(range(50))
)

# Building the Oracle

In [0]:
oracle = Sequential()

In [0]:
input_layer = Dense(
    units=856,
    activation='relu',
    input_dim=856,
)

In [0]:
hidden_layer = Dense(
    units=30,
    activation='relu',
)

In [0]:
output_layer = Dense(
    units=50,
    activation='softmax',
)

In [17]:
oracle.add(input_layer)
oracle.add(hidden_layer)
oracle.add(output_layer)

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
oracle.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'],
)

# Training Oracle on Dataset

In [19]:
oracle.fit(training_data,  training_labels, epochs=3, batch_size=50)

Instructions for updating:
Use tf.cast instead.
Epoch 1/3
220545/220545 [==============================] - 90s 409us/step - loss: 0.1862 - acc: 0.9465
Epoch 2/3
220545/220545 [==============================] - 91s 413us/step - loss: 0.0664 - acc: 0.9776
Epoch 3/3
220545/220545 [==============================] - 95s 433us/step - loss: 0.0509 - acc: 0.9820


In [20]:
oracle

# Evaluating the Oracle

In [21]:
loss, accuracy = oracle.evaluate(testing_data, testing_labels)

24505/24505 [==============================] - 2s 82us/step


In [22]:
loss, accuracy

(0.05585555036787908, 0.9827382166685802)

In [23]:
oracle.metrics_names

['loss', 'acc']

# Building a Substitute Model

Attack Model:
  - We only have access to the oracle as a black box, allowing the following interactions:
    - Send input.
    - Receive a prediction.

Need to search the input space to find the decision boundaries, use these inputs to train the substitute model. Note: we can overfit here and it's absolutely fine!

The original "Blackbox... " paper has an accompanying Python library,  [cleverhans](https://github.com/tensorflow/cleverhans/), which we use to implement this attack.

First, mirror the architecture of the oracle:
    

In [0]:
substitute = Sequential()

input_layer = Dense(
    units=856,
    activation='relu',
    input_dim=856,
)
hidden_layer = Dense(
    units=30,
    activation='relu',
)
output_layer = Dense(
    units=50,
    activation='softmax',
)

substitute.add(input_layer)
substitute.add(hidden_layer)
substitute.add(output_layer)

# We need to convert our substitute model into the cleverhans format.
substitute_ch = KerasModelWrapper(substitute)

In [0]:
tensorflow_session = tensorflow.Session()

We start by giving the adversary a small dataset with which to bootstrap it's search. Initially, we give it a random sample of 5% of the original data set. 

We can then steal the rest of the dataset to determine the accuracy of our substitute model. **NOTE** They do this in the tutorial code but, is it legit? Or are we cheating?


In [26]:
adversary_training_set, adversary_test_set = train_test_split(labelled_dataset, train_size=0.05)

adversary_training_inputs = adversary_training_set.drop('user', axis='columns')
adversary_training_labels = adversary_training_set['user'] - 1  # keras requires 0 based index

# For some reason cleverhans doesn't detect a GPU when it runs, but our models at the top using
# keras _do_. I think this creates a type mis-match: code running on the GPU uses numpy.float64
# whilst the cleverhans stuff runs on the CPU and extects numpy.float32 (or vica versa).
#   -> This is why this dodgy type conversion exists:
adversary_training_inputs = adversary_training_inputs.values.astype(numpy.float32)
adversary_training_labels = adversary_training_labels.values

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Define input placeholders for the tensor flow model (these are then used to generate new points)

In [0]:
number_of_users = 50
number_of_commands = 856

input_placeholder = tensorflow.placeholder(
    tensorflow.float32,
    shape=(None, number_of_commands)
)

output_placeholder = tensorflow.placeholder(
    tensorflow.float32,
    shape=(None, number_of_users)
)

Get the oracles predictions for the "bootstrap" inputs:

In [0]:
bootstrap_oracle_predictions = oracle.predict(adversary_training_inputs)

Train substitute using method from https://arxiv.org/abs/1602.02697

In [0]:
# print("Training the substitute model.")
# model_sub, preds_sub  = train_sub(
#     tensorflow_session,
#     input_placeholder,
#     output_placeholder, 
#     bootstrap_oracle_predictions,  # predictions from the blackbox model for the _adversary_ training set?
#     adversary_training_inputs,
#     keras.utils.to_categorical(adversary_training_labels, num_classes=50),
#     number_of_users,  # number of outputs
#     10, # epochs
#     32, # batch size
#     0.001,  # learning rate (I copied and pasted default from here: https://www.tensorflow.org/api_docs/python/tf/train/AdamOptimizer)
#     6, # Number of substitute data augmentations, 6 frm code
#     0.1,  # step-size of the jacobian augmentation, 0.1 from paper
#     512, # aug batch size
#     rng, # an rng
#     number_of_commands,  # number of input features
# )

In [0]:
# Creates the substitute by alternatively augmenting the training data and training the substitute.
#   :param sess: TF session
#   :param x: input TF placeholder
#   :param y: output TF placeholder
#   :param bbox_preds: output of black-box model predictions
#   :param x_sub: initial substitute training data
#   :param y_sub: initial substitute training labels
#   :param nb_classes: number of output classes
#   :param nb_epochs_s: number of epochs to train substitute model
#   :param batch_size: size of training batches
#   :param learning_rate: learning rate for training
#   :param data_aug: number of times substitute training data is augmented
#   :param lmbda: lambda from arxiv.org/abs/1602.02697
#   :param rng: numpy.random.RandomState instance

In [0]:
# Define the predictions and loss of the model, symbolically in TensorFlow (i.e. these variables 
# point to the result of calculations that haven't been performed yet)
substitute_predictions = substitute_ch.get_logits(input_placeholder)
substitute_loss = CrossEntropy(substitute_ch, smoothing=0)

Here we define a Jacobian Graph/Model (**TODO** What in the world is this? and why are we using it?).

In [32]:
adversary_training_inputs.shape, adversary_training_labels.shape

((12252, 856), (12252,))

In [33]:
# Define the Jacobian symbolically using TensorFlow
grads = jacobian_graph(substitute_predictions, input_placeholder, number_of_users)

number_of_dataset_augmentation_batches = 5
dataset_augmentation_batch_size = 512


lmbda = 1  # this is the step-size of the Jacobian augmentation (tutorial used 0.1, but we are 
                      # working in ints so try 1).


# Train the substitute and augment dataset alternatively
for batch in range(number_of_dataset_augmentation_batches):
    print("BATCH #" + str(batch))
    
    print("Substitute training epoch:")
    train(
        tensorflow_session, 
        substitute_loss,
        adversary_training_inputs, 
        keras.utils.to_categorical(adversary_training_labels, num_classes=50),
        init_all=False,
        args={
            'nb_epochs': 10,  # copied from tutorial (may need tweaking)
            'batch_size': 32,  # copied from tutorial (may need tweaking)
            'learning_rate': 0.001,  # copied from tutorial (may need tweaking)
        },
        rng=rng,
        # var_list=  # list of model parameters to train (optional so left out for now)
    )
    

    # If we are not at last substitute training iteration, augment dataset
    in_final_batch = batch == number_of_dataset_augmentation_batches - 1
    if not in_final_batch:
        print("Generating new data points:")
        
        # Use Jacobian augmentation to generate new data points:
        
        # TODO: What is this?? I think this scales importance per batch? != 0 means they are
        # alternating mod 3 I think.
        lmbda_coef = 2 * int(int(batch / 3) != 0) - 1 

        augmented_dataset_inputs = jacobian_augmentation(
            tensorflow_session, 
            input_placeholder, 
            adversary_training_inputs, 
            adversary_training_labels,
            grads,
            lmbda_coef * lmbda,
            dataset_augmentation_batch_size,
        )

        # Send the newly generated data points to the oracle, and use its output as their labels:
        augmented_dataset_labels = oracle.predict(augmented_dataset_inputs)
        just_new_labels = augmented_dataset_labels[len(adversary_training_inputs):]

        # Note here that we take the argmax because the adversary
        # only has access to the label (not the probabilities) output
        # by the black-box model
        thresholded_new_labels = numpy.argmax(just_new_labels, axis=1)

        augmented_dataset_labels = numpy.hstack([adversary_training_labels, thresholded_new_labels])

        # Replace dataset and labels with augmented dataset and labels
        adversary_training_inputs = augmented_dataset_inputs
        adversary_training_labels = augmented_dataset_labels

BATCH #0
Substitute training epoch:
Instructions for updating:
dim is deprecated, use axis instead
num_devices:  1


/usr/local/lib/python3.6/dist-packages/cleverhans/utils_tf.py:511: UserWarning: No GPUS, running on CPU
  warnings.warn("No GPUS, running on CPU")
[INFO 2019-03-15 11:50:25,881 cleverhans] Epoch 0 took 2.216188430786133 seconds
[INFO 2019-03-15 11:50:28,188 cleverhans] Epoch 1 took 2.0956947803497314 seconds
[INFO 2019-03-15 11:50:30,596 cleverhans] Epoch 2 took 2.186173439025879 seconds
[INFO 2019-03-15 11:50:32,904 cleverhans] Epoch 3 took 2.10575795173645 seconds
[INFO 2019-03-15 11:50:35,368 cleverhans] Epoch 4 took 2.259108066558838 seconds
[INFO 2019-03-15 11:50:37,736 cleverhans] Epoch 5 took 2.159675359725952 seconds
[INFO 2019-03-15 11:50:40,124 cleverhans] Epoch 6 took 2.171527862548828 seconds
[INFO 2019-03-15 11:50:42,556 cleverhans] Epoch 7 took 2.217026710510254 seconds
[INFO 2019-03-15 11:50:44,868 cleverhans] Epoch 8 took 2.1028528213500977 seconds
[INFO 2019-03-15 11:50:47,315 cleverhans] Epoch 9 took 2.2374422550201416 seconds


Generating new data points:
BATCH #1
Substitute training epoch:
num_devices:  1


[INFO 2019-03-15 11:51:21,674 cleverhans] Epoch 0 took 4.670348167419434 seconds
[INFO 2019-03-15 11:51:26,514 cleverhans] Epoch 1 took 4.274680852890015 seconds
[INFO 2019-03-15 11:51:31,397 cleverhans] Epoch 2 took 4.329198837280273 seconds
[INFO 2019-03-15 11:51:36,048 cleverhans] Epoch 3 took 4.098728656768799 seconds
[INFO 2019-03-15 11:51:40,846 cleverhans] Epoch 4 took 4.233973264694214 seconds
[INFO 2019-03-15 11:51:45,575 cleverhans] Epoch 5 took 4.163672685623169 seconds
[INFO 2019-03-15 11:51:50,582 cleverhans] Epoch 6 took 4.459063291549683 seconds
[INFO 2019-03-15 11:51:55,424 cleverhans] Epoch 7 took 4.290295124053955 seconds
[INFO 2019-03-15 11:52:00,353 cleverhans] Epoch 8 took 4.373022079467773 seconds
[INFO 2019-03-15 11:52:05,211 cleverhans] Epoch 9 took 4.304365634918213 seconds


Generating new data points:
BATCH #2
Substitute training epoch:
num_devices:  1


[INFO 2019-03-15 11:53:20,451 cleverhans] Epoch 0 took 9.779504776000977 seconds
[INFO 2019-03-15 11:53:30,450 cleverhans] Epoch 1 took 8.863205432891846 seconds
[INFO 2019-03-15 11:53:41,004 cleverhans] Epoch 2 took 9.429665565490723 seconds
[INFO 2019-03-15 11:53:50,575 cleverhans] Epoch 3 took 8.448460102081299 seconds
[INFO 2019-03-15 11:54:00,170 cleverhans] Epoch 4 took 8.473943710327148 seconds
[INFO 2019-03-15 11:54:10,507 cleverhans] Epoch 5 took 9.213356256484985 seconds
[INFO 2019-03-15 11:54:20,309 cleverhans] Epoch 6 took 8.677577257156372 seconds
[INFO 2019-03-15 11:54:30,726 cleverhans] Epoch 7 took 9.291094064712524 seconds
[INFO 2019-03-15 11:54:41,491 cleverhans] Epoch 8 took 9.63745379447937 seconds
[INFO 2019-03-15 11:54:51,303 cleverhans] Epoch 9 took 8.685791492462158 seconds


Generating new data points:
BATCH #3
Substitute training epoch:
num_devices:  1


[INFO 2019-03-15 11:57:11,223 cleverhans] Epoch 0 took 16.66619610786438 seconds
[INFO 2019-03-15 11:57:30,133 cleverhans] Epoch 1 took 16.351417541503906 seconds
[INFO 2019-03-15 11:57:49,304 cleverhans] Epoch 2 took 16.861384391784668 seconds
[INFO 2019-03-15 11:58:09,333 cleverhans] Epoch 3 took 17.724542379379272 seconds
[INFO 2019-03-15 11:58:28,476 cleverhans] Epoch 4 took 16.73256754875183 seconds
[INFO 2019-03-15 11:58:47,202 cleverhans] Epoch 5 took 16.389927864074707 seconds
[INFO 2019-03-15 11:59:06,470 cleverhans] Epoch 6 took 16.961921453475952 seconds
[INFO 2019-03-15 11:59:26,538 cleverhans] Epoch 7 took 17.73899555206299 seconds
[INFO 2019-03-15 11:59:46,042 cleverhans] Epoch 8 took 17.21279287338257 seconds
[INFO 2019-03-15 12:00:05,684 cleverhans] Epoch 9 took 17.309788942337036 seconds


Generating new data points:
BATCH #4
Substitute training epoch:
num_devices:  1


[INFO 2019-03-15 12:04:58,472 cleverhans] Epoch 0 took 32.7903847694397 seconds
[INFO 2019-03-15 12:05:37,430 cleverhans] Epoch 1 took 33.580817461013794 seconds
[INFO 2019-03-15 12:06:18,897 cleverhans] Epoch 2 took 36.45302939414978 seconds
[INFO 2019-03-15 12:06:58,457 cleverhans] Epoch 3 took 34.504377603530884 seconds
[INFO 2019-03-15 12:07:37,082 cleverhans] Epoch 4 took 33.607105016708374 seconds
[INFO 2019-03-15 12:08:15,253 cleverhans] Epoch 5 took 33.17499923706055 seconds
[INFO 2019-03-15 12:08:54,352 cleverhans] Epoch 6 took 34.100393533706665 seconds
[INFO 2019-03-15 12:09:32,793 cleverhans] Epoch 7 took 33.41313028335571 seconds
[INFO 2019-03-15 12:10:11,542 cleverhans] Epoch 8 took 33.7249174118042 seconds
[INFO 2019-03-15 12:10:50,635 cleverhans] Epoch 9 took 34.07659983634949 seconds


In [0]:
#                )\         O_._._._A_._._._O         /(               
#                 \`--.___,'=================`.___,--'/                
#                  \`--._.__                 __._,--'/                 
#                    \  ,. l`~~~~~~~~~~~~~~~'l ,.  /                   
#        __            \||(_)!_!_!_.-._!_!_!(_)||/            __       
#        \\`-.__        ||_|____!!_|;|_!!____|_||        __,-'//       
#         \\    `==---='-----------'='-----------`=---=='    //        
#         | `--.                _   _   _                ,--' |        
#         | `--.               / \ / \ / \               ,--' |        
#         | `--.          ~~~ ( R | A | M ) ~~~          ,--' |        
#         | `--.               \_/ \_/ \_/               ,--' |        
#          \  ,.`~~~~~~~~~~~~~             ~~~~~~~~~~~~~',.  /         
#            \||  ____,-------._,-------._,-------.____  ||/           
#             ||\|___!`======="!`======="!`======="!___|/||            
#             || |---||--------||-| | |-!!--------||---| ||            
#   __O_____O_ll_lO_____O_____O|| |'|'| ||O_____O_____Ol_ll_O_____O__  
#   o H o o H o o H o o H o o |-----------| o o H o o H o o H o o H o  
#  ___H_____H_____H_____H____O =========== O____H_____H_____H_____H___ 
#                           /|=============|\                          
# ()______()______()______() '==== +-+ ====' ()______()______()______()
# ||{_}{_}||{_}{_}||{_}{_}/| ===== |_| ===== |\{_}{_}||{_}{_}||{_}{_}||
# ||      ||      ||     / |==== s(   )s ====| \     ||      ||      ||
# ======================()  =================  ()======================
# ----------------------/| ------------------- |\----------------------
#                      / |---------------------| \                     
# -'--'--'           ()  '---------------------'  ()                   
#                    /| ------------------------- |\    --'--'--'      
#        --'--'     / |---------------------------| \    '--'          
#                 ()  |___________________________|  ()           '--'-
#   --'-          /| _______________________________  |\               
#  --' gpyy      / |__________________________________| \           

# ALL HAIL THE RAM GODS
# We used tons of memory in the above step, so delete everything
# we don't need and manually run the GC.

# TODO

del augmented_dataset_inputs
del augmented_dataset_labels

In [35]:
adversary_training_labels

array([41, 36, 48, ...,  3, 19, 43])

In [0]:
adversary_test_inputs = adversary_test_set.drop('user', axis='columns')
adversary_test_labels = adversary_test_set['user'] - 1  # keras requires 0 based index

# For some reason cleverhans doesn't detect a GPU when it runs, but our models at the top using
# keras _do_. I think this creates a type mis-match: code running on the GPU uses numpy.float64
# whilst the cleverhans stuff runs on the CPU and extects numpy.float32 (or vica versa).
#   -> This is why this dodgy type conversion exists:
adversary_test_inputs = adversary_test_inputs.values.astype(numpy.float32)
adversary_test_labels = adversary_test_labels.values

In [0]:
from cleverhans.utils_tf import model_eval

adversary_test_labels_one_hot = keras.utils.to_categorical(adversary_test_labels, num_classes=50)

# Evaluate the substitute model on clean test examples
acc = model_eval(
    tensorflow_session, 
    input_placeholder,
    output_placeholder,
    substitute_predictions,
    adversary_test_inputs,
    adversary_test_labels_one_hot,
    args={'batch_size': 32}
)

In [38]:
acc

0.9553475545322554

Just inspecting the generated dataset. Notes:
  1. Some of the values are negative!
  2. The real dataset has an input range of 0-100. This search technique has found all of them, plus a few on each side.
  3. The augmented dataset has just less than 200,000 data points. That's almost as many as were used to train the oracle.

In [39]:
numpy.unique(adversary_training_inputs)

array([ -4.,  -3.,  -2.,  -1.,   0.,   1.,   2.,   3.,   4.,   5.,   6.,
         7.,   8.,   9.,  10.,  11.,  12.,  13.,  14.,  15.,  16.,  17.,
        18.,  19.,  20.,  21.,  22.,  23.,  24.,  25.,  26.,  27.,  28.,
        29.,  30.,  31.,  32.,  33.,  34.,  35.,  36.,  37.,  38.,  39.,
        40.,  41.,  42.,  43.,  44.,  45.,  46.,  47.,  48.,  49.,  50.,
        51.,  52.,  53.,  54.,  55.,  56.,  57.,  58.,  59.,  60.,  61.,
        62.,  63.,  64.,  65.,  66.,  67.,  68.,  69.,  70.,  71.,  72.,
        73.,  74.,  75.,  76.,  77.,  78.,  79.,  80.,  81.,  82.,  83.,
        84.,  85.,  86.,  87.,  88.,  89.,  90.,  91.,  92.,  93.,  94.,
        95.,  96.,  97.,  98.,  99., 100., 101.], dtype=float32)

In [40]:
len(adversary_training_inputs)

196032

In [41]:
pandas.DataFrame(adversary_training_inputs[numpy.random.choice(adversary_training_inputs.shape[0], size=20)])

,0,1,2,3,4,5,6,7,8,9,...,846,847,848,849,850,851,852,853,854,855
0,-1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,...,-1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0
1,-2.0,2.0,-2.0,0.0,0.0,0.0,2.0,0.0,-2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0
2,-1.0,1.0,1.0,-1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,...,-1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0
3,-1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,...,-1.0,-1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0
4,1.0,-1.0,-1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,...,1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0
5,2.0,0.0,-2.0,-2.0,-2.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,2.0,0.0,2.0,2.0,2.0,-2.0,2.0,2.0
6,-2.0,2.0,2.0,-2.0,0.0,0.0,2.0,0.0,-2.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,-2.0,2.0,0.0,0.0
7,-1.0,3.0,3.0,-1.0,1.0,-1.0,-1.0,-1.0,-3.0,-1.0,...,-1.0,1.0,3.0,1.0,-3.0,-1.0,-3.0,-1.0,-1.0,-1.0
8,-2.0,2.0,-2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
9,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,-2.0,0.0,0.0,0.0,0.0


# Crafting Adversarial Examples (TODO)

In [0]:
# Initialize the Fast Gradient Sign Method (FGSM) attack object.
fgsm_par = {'eps': 1., 'ord': numpy.inf, 'clip_min': 0., 'clip_max': 100.}
fgsm = FastGradientMethod(substitute_ch, sess=tensorflow_session)

In [43]:
# Craft adversarial examples using the substitute
eval_params = {'batch_size': dataset_augmentation_batch_size}
x_adv_sub = fgsm.generate(input_placeholder, **fgsm_par)

Instructions for updating:
Use tf.cast instead.


In [44]:
x_adv_sub

<tf.Tensor 'Identity:0' shape=(?, 856) dtype=float32>

In [0]:
oracle_keras = KerasModelWrapper(oracle)
oracle_fgsm_pred = oracle_keras.get_logits(x_adv_sub)

In [46]:
oracle_fgsm_pred

<tf.Tensor 'model_2/dense_3/BiasAdd:0' shape=(?, 50) dtype=float32>

In [47]:
# Evaluate the accuracy of the "black-box" model on adversarial examples
accuracy = model_eval(
        tensorflow_session,
        input_placeholder,
        output_placeholder,
        oracle_fgsm_pred,
        adversary_test_inputs,
        adversary_test_labels_one_hot,
        args=eval_params
        )
print('Test accuracy of oracle on adversarial examples generated '
    'using the substitute: ' + str(accuracy))

Test accuracy of oracle on adversarial examples generated using the substitute: 0.0160267699894329


So we fooled it! Yay

### Targetting on user 16

Pick an example command vector which wasn't user 16, then generate a similar example that is classified as user 16. 

We then put that example into the oracle model and check it's classification. If it is classified as user 16, our attempt was successfull.

In [48]:
original_command_vectors = numpy.array([adversary_test_inputs[0]])
original_labels = numpy.array([adversary_test_labels_one_hot[0]])

target_labels = keras.utils.to_categorical(numpy.array([16]), num_classes=50)

# TODO: Make new attack class, and override +wrap FastGradientMethod.optimize_linear
# so that perturbations are min(0, optimal perturbation). This will give additive only searches.

attack = FastGradientMethod(substitute_ch, sess=tensorflow_session)
adversarial_examples = attack.generate_np(
    original_command_vectors,
    y_target=target_labels,
    eps=1.0,
    ord=numpy.inf,
    clip_min=0.0,
    clip_max=100.0,
)

predicted_labels = oracle.predict(adversarial_examples)

[INFO 2019-03-15 12:11:47,283 cleverhans] Constructing new graph for attack FastGradientMethod


In [49]:
adversary_test_labels[0], predicted_labels[0][16]

(26, 0.9999511)

In [0]:
adversary_test_labels[0], predicted_labels[0][16]

(26, 1.0)

In [0]:
original_command_vectors = numpy.array([adversary_test_inputs[0]])
original_labels = numpy.array([adversary_test_labels_one_hot[0]])

target_labels = keras.utils.to_categorical(numpy.array([16]), num_classes=50)

adversarial_examples = fgsm.generate_np(
    original_command_vectors,
    y_target=target_labels,
    eps=1.0,
    ord=numpy.inf,
    clip_min=0.0,
    clip_max=100.0,
)

predicted_labels = oracle.predict(adversarial_examples)

smm_attack = SaliencyMapMethod(substitute_ch, sess=tensorflow_session)

[INFO 2019-03-14 18:04:30,721 cleverhans] Constructing new graph for attack SaliencyMapMethod


In [0]:
adversary_test_labels[1], predicted_labels[0][20]

(20, 0.99999523)

In [0]:
adversarial_examples

array([[12.599993,  0.1     ,  0.1     ,  0.1     ,  0.1     ,  0.1     ,
         0.1     ,  0.1     ,  0.1     ,  0.1     ,  1.      ,  0.1     ,
         0.1     ,  0.1     ,  0.      ,  0.      ,  0.1     ,  0.1     ,
         0.1     ,  0.1     ,  0.1     ,  0.1     ,  0.1     ,  0.1     ,
         0.1     ,  0.1     ,  0.1     ,  0.1     ,  0.1     ,  0.1     ,
         0.1     ,  0.1     ,  0.1     ,  0.1     ,  0.1     ,  0.1     ,
         0.1     ,  0.1     ,  0.1     ,  0.1     ,  0.1     ,  0.1     ,
         0.      ,  0.      ,  0.1     ,  0.1     ,  0.1     ,  0.      ,
         0.1     ,  0.1     ,  0.      ,  0.1     ,  0.1     ,  0.1     ,
         0.1     ,  0.1     ,  0.1     ,  0.      ,  0.1     ,  0.      ,
         0.1     ,  0.1     ,  0.1     ,  0.1     ,  0.      ,  0.1     ,
         0.1     ,  0.1     ,  0.1     ,  0.1     ,  0.1     ,  0.1     ,
         0.      ,  6.1     ,  0.1     ,  0.1     ,  0.1     ,  0.1     ,
         0.      ,  0.      ,  0.1    

In [0]:
user16_target = numpy.concatenate((numpy.zeros(15), [1], numpy.zeros(34)))
user16_target

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [0]:
fgsm_par_targetted = {'eps': 1., 'ord': numpy.inf, 'clip_min': 0., 'clip_max': 100., 'y_target': numpy.array([user16_target])}

In [0]:
x_adv_sub_targetted = fgsm.generate(input_placeholder, **fgsm_par_targetted)

In [0]:
oracle_fgsm_pred_targetted = oracle_keras.get_logits(x_adv_sub_targetted)

accuracy_targetted = model_eval(
                tensorflow_session,
                input_placeholder,
                output_placeholder,
                oracle_fgsm_pred_targetted,
                numpy.array([adversary_test_inputs[0]]),
                numpy.array([adversary_test_labels_one_hot[0]]),
                args=eval_params
                )
print('Test accuracy of oracle on adversarial example with a target of User 16 generated '
    'using the substitute: ' + str(accuracy))

Test accuracy of oracle on adversarial example with a target of User 16 generated using the substitute: 0.016190001632316427


In [0]:
help(oracle_fgsm_pred_targetted.eval)

Help on method eval in module tensorflow.python.framework.ops:

eval(feed_dict=None, session=None) method of tensorflow.python.framework.ops.Tensor instance
    Evaluates this tensor in a `Session`.
    
    Calling this method will execute all preceding operations that
    produce the inputs needed for the operation that produces this
    tensor.
    
    *N.B.* Before invoking `Tensor.eval()`, its graph must have been
    launched in a session, and either a default session must be
    available, or `session` must be specified explicitly.
    
    Args:
      feed_dict: A dictionary that maps `Tensor` objects to feed values.
        See `tf.Session.run` for a
        description of the valid feed values.
      session: (Optional.) The `Session` to be used to evaluate this tensor. If
        none, the default session will be used.
    
    Returns:
      A numpy array corresponding to the value of this tensor.

